# Bay Wheels's trip Dataset Exploration
## by Chrysanthi Polyzoni

> We will be exploring data teken from [Bay Wheels Bike share service](https://www.lyft.com/bikes/bay-wheels/system-data). Bay Wheels offers affordable, accessible, and fun transportation option for everyone. [Bay Area](https://it.wikipedia.org/wiki/San_Francisco_Bay_Area) residents who qualify for CalFresh, SFMTA Lifeline Pass, or PG&E CARE utility discount are eligible to join Bike Share for All program for 5 USD for the first year — now accepting prepaid cards!

The Data
Each trip is anonymized and includes:

* Trip Duration (seconds)
* Start Time and Date
* End Time and Date
* Start Station ID
* Start Station Name
* Start Station Latitude
* Start Station Longitude
* End Station ID
* End Station Name
* End Station Latitude
* End Station Longitude
* Bike ID
* User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual

This data is provided according to the [Bay Wheels License Agreement](https://baywheels-assets.s3.amazonaws.com/data-license-agreement.html).

<a id='top'></a>
## Preliminary Wrangling

- [Gathering the Data](#aquire)
- [Assessing](#assess)
- [Cleaning](#clean)
- [Exploratory Data Analysis](#analyze)

<a id='aquire'></a>
## Gathering the Data for the greater San Fransisco Bay Area:
The Data for one month of a year is saved in a zipped 'csv' File on the [Website of the Company](https://s3.amazonaws.com/baywheels-data/index.html) . As an example, the File '201801-fordgobike-tripdata.csv.zip' contains all the rides that occured in January 2018.
After programmatically downloading and unzipping all available months of the year 2018,
We will load and concatenate all <i>CSV</i> Files in a Dataframe where we will be conducting our analysis on.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.

Downloading all necessary Files for the year 2019 programmatically:¶

In [2]:
import requests
import os
import zipfile

csv_downloaded = False # Set to True to prevent downloading Process from occuring \
                      #every time we re-run the notebook
filename_ford = '2019xx-fordgobike-tripdata.csv.zip'
filename_wheels = '2019xx-baywheels-tripdata.csv.zip'
base_url = 'https://s3.amazonaws.com/baywheels-data/'

datafolder = './data' # keeping all csv files in a 'data' directory, under the current directory
if not os.path.exists(datafolder):
    os.makedirs(datafolder)
    print('make {}'.format(datafolder))

for i in range(5, 13):
    if csv_downloaded:
        break
    
    iter_wheels = str(filename_wheels).replace("xx", "{:02d}".format(i))
    
    url_wheels = base_url + iter_wheels
    
    local_wheels = os.path.join(datafolder, iter_wheels)
    
    # download the zip file locally
    if not os.path.exists(local_wheels):
        print("{} does not exist. --> download it".format(local_wheels))
        try:
            response = requests.get(url_wheels)
            with open(local_wheels, mode='wb') as file:
                file.write(response.content)
                    # unzip file to the previously created 'data' directory
                with zipfile.ZipFile(local_wheels, 'r') as myzip:
                    myzip.extractall(path = datafolder)
                    print("{} --> unzipping file".format(local_wheels))
            os.remove(local_wheels)
            print("{}--> removing initial zip file".format(local_wheels))
        except OSError as err:
            print("OS error: {0}".format(err))
            
for i in range(1, 5):
    if csv_downloaded:
        break
    iter_ford = str(filename_ford).replace("xx", "{:02d}".format(i))
    url_ford = base_url + iter_ford
    local_ford = os.path.join(datafolder, iter_ford)

            
    if not os.path.exists(local_ford):
        print("{} does not exist. --> download it".format(local_ford))
        try:
            response = requests.get(url_ford)
            with open(local_ford, mode='wb') as file:
                file.write(response.content)
                    # unzip file to the previously created 'data' directory
                with zipfile.ZipFile(local_ford, 'r') as myzip:
                    myzip.extractall(path = datafolder)
                    print("{} --> unzipping file".format(local_ford))
            os.remove(local_ford)
            print("{}--> removing initial zip file".format(local_ford))
        except OSError as err:
            print("OS error: {0}".format(err))

os.listdir(datafolder)

make ./data
./data\201905-baywheels-tripdata.csv.zip does not exist. --> download it
./data\201905-baywheels-tripdata.csv.zip --> unzipping file
./data\201905-baywheels-tripdata.csv.zip--> removing file
./data\201906-baywheels-tripdata.csv.zip does not exist. --> download it
./data\201906-baywheels-tripdata.csv.zip --> unzipping file
./data\201906-baywheels-tripdata.csv.zip--> removing file
./data\201907-baywheels-tripdata.csv.zip does not exist. --> download it
./data\201907-baywheels-tripdata.csv.zip --> unzipping file
./data\201907-baywheels-tripdata.csv.zip--> removing file
./data\201908-baywheels-tripdata.csv.zip does not exist. --> download it
./data\201908-baywheels-tripdata.csv.zip --> unzipping file
./data\201908-baywheels-tripdata.csv.zip--> removing file
./data\201909-baywheels-tripdata.csv.zip does not exist. --> download it
./data\201909-baywheels-tripdata.csv.zip --> unzipping file
./data\201909-baywheels-tripdata.csv.zip--> removing file
./data\201910-baywheels-tripdata.

['201901-fordgobike-tripdata.csv',
 '201902-fordgobike-tripdata.csv',
 '201903-fordgobike-tripdata.csv',
 '201904-fordgobike-tripdata.csv',
 '201905-baywheels-tripdata.csv',
 '201906-baywheels-tripdata.csv',
 '201907-baywheels-tripdata.csv',
 '201908-baywheels-tripdata.csv',
 '201909-baywheels-tripdata.csv',
 '201910-baywheels-tripdata.csv',
 '201911-baywheels-tripdata.csv',
 '201912-baywheels-tripdata.csv',
 '__MACOSX']

In [54]:
%%time

df_list = [] # list of dataframes to read the csv files in.
x= 0

for file in os.listdir(datafolder):
    if file.endswith('.csv'):
        col_list = ['duration_sec', 'start_time', 'end_time', 'start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type']
        df_name = 'df_' + str(file[4:6])
        df_temp = pd.read_csv('./data/' + file)
        print('\nReading {} into {} Pandas Data Frame with dimensions {}'.format(file, df_name, df_temp.shape))
        df_features = list(df_temp.columns)
        #print(df_features)
        for col in df_temp.columns:
            if col not in col_list:
                df_temp.drop(col, axis=1, inplace = True)
        print('Updated shape after dropping non consistent features in all DataFrames is {}'.format(df_temp.shape))
        df_list.append(df_temp)
        x += df_temp.shape[0]

print('\nNumber of dfs read: ', len(df_list))
print('Number of rows read: ', x)


Reading 201901-fordgobike-tripdata.csv into df_01 Pandas Data Frame with dimensions (192082, 14)
Updated shape after dropping non consistent features in all DataFrames is (192082, 13)

Reading 201902-fordgobike-tripdata.csv into df_02 Pandas Data Frame with dimensions (183412, 14)
Updated shape after dropping non consistent features in all DataFrames is (183412, 13)

Reading 201903-fordgobike-tripdata.csv into df_03 Pandas Data Frame with dimensions (256299, 14)
Updated shape after dropping non consistent features in all DataFrames is (256299, 13)

Reading 201904-fordgobike-tripdata.csv into df_04 Pandas Data Frame with dimensions (239111, 14)
Updated shape after dropping non consistent features in all DataFrames is (239111, 13)

Reading 201905-baywheels-tripdata.csv into df_05 Pandas Data Frame with dimensions (182163, 14)
Updated shape after dropping non consistent features in all DataFrames is (182163, 13)

Reading 201906-baywheels-tripdata.csv into df_06 Pandas Data Frame with dim

### What is the structure of your dataset?

> My dataset has 2 506 983 reccords (in other words attributes or lines) and 13 columns

### What is/are the main feature(s) of interest in your dataset?

> * Trip Duration (seconds) `duration_sec`
* Start Time and Date `start_time`
* End Time and Date `end_time`
* Start Station ID `start_station_id`
* Start Station Name `start_station_name`
* Start Station Latitude `start_station_latitude`
* Start Station Longitude `start_station_longitude`
* End Station ID `end_station_id`
* End Station Name `end_station_name`
* End Station Latitude `end_station_latitude`
* End Station Longitude `end_station_longitude`
* Bike ID `bike_id`
* User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual `user_type`


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> * Trip Duration (seconds) `duration_sec`
* Start Time and Date `start_time`
* End Time and Date `end_time`
* Start Station ID `start_station_id`
* Start Station Name `start_station_name`
* Start Station Latitude `start_station_latitude`
* Start Station Longitude `start_station_longitude`
* End Station ID `end_station_id`
* End Station Name `end_station_name`
* End Station Latitude `end_station_latitude`
* End Station Longitude `end_station_longitude`
* Bike ID `bike_id`
* User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual `user_type`

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!